In [1]:

import os
# import random
# import shutil
# import time
import warnings
import sys
sys.path.append('../')

import torch
import torch.nn as nn
# import torch.nn.parallel
# import torch.backends.cudnn as cudnn
# import torch.distributed as dist

import torch.optim
# import torch.multiprocessing as mp
import torch.utils.data
# import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
# import torchvision.models as models
from resnet import *

from main import *


In [2]:
import easydict 
args = easydict.EasyDict({ "batch_size": 8, 
                          "epochs": 50, 
                          "data": 0, 
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'workers':4,
                         'output_dir' :'../trained_models/retinanet/'})

In [3]:
backbone_model= torchvision.models.resnet101(pretrained=True)
model = resnet101(pretrained=False)
print(model.conv1.weight[0,0,0])
model.load_state_dict(backbone_model.state_dict(), strict=False)
print(model.conv1.weight[0,0,0])

# num_ftrs = model.fc1.in_features
# print(num_ftrs)
# model.fc1 = nn.Linear(num_ftrs, 2) # target label is 2
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)

device = torch.device('cuda')
model.to(device)
criterion = criterion.to(device)

tensor([ 0.0089, -0.0306, -0.0410, -0.0098, -0.0068,  0.0341, -0.0089],
       grad_fn=<SelectBackward>)
tensor([ 0.0202, -0.0044, -0.0183, -0.0284, -0.0152, -0.0016,  0.0093],
       grad_fn=<SelectBackward>)


In [4]:
# Data loading code
data_dir = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [6]:
# next(iter(train_dataset))

In [7]:
# train_sampler = torch.utils.data.distributed.DistributedSampler(train_dataset)
train_sampler = None

In [8]:
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=64, shuffle=(train_sampler is None),
    num_workers=8, pin_memory=True, sampler=train_sampler)

In [9]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, shuffle=False,
    num_workers=8, pin_memory=True)

In [10]:
# model.to(device)

In [ ]:
best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []
for epoch in range(args.start_epoch, args.epochs):
    adjust_learning_rate(optimizer, epoch, args)

    # train for one epoch
    epoch_loss = train(train_loader, model, criterion, optimizer, epoch, args)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, args)  
    
    train_loss.append(epoch_loss)
    val_acc.append(acc1)
    print('************train_loss {} val_acc {}*************'.format(epoch_loss, acc1))
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

#     if not args.multiprocessing_distributed or (args.multiprocessing_distributed
#             and args.rank % ngpus_per_node == 0):
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best, args, filename='../trained_model/checkpoint.pt')    

/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][    0/20019]	Time  1.165 ( 1.165)	Data  0.746 ( 0.746)	Loss 7.0750e+00 (7.0750e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.00 (  0.00)
